<a href="https://colab.research.google.com/github/anruki/Procesamiento_YOLO/blob/main/p3_Miguel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install opencv-python ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 24.5 MB/s eta 0:00:00


In [3]:
from ultralytics import YOLO
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
from google.colab.patches import cv2_imshow


##EJERCICI0 1

CABALLO FUERA DEL RECINTO

In [12]:
# Cargar modelo YOLOv8
model = YOLO('yolov8n.pt')

# Ruta del video de entrada
video_path = 'caballo.mp4'

# Cargar el video
cap = cv2.VideoCapture(video_path)

# CREAR VIDEO PROCESADO (OUTPUT)
#------------------------------------------------------------------
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'caballo_procesado_caballos.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

ret = True
#------------------------------------------------------------------

# PROCESAMIENTO
while ret:
    ret, frame = cap.read()
    if ret:
        # TRACKLING
        results = model.track(frame, persist=True)

        # DETECTAR SOLO CABALLOS
        for result in results[0].boxes:
            label = model.names[int(result.cls)]
            if label == 'horse':
                x1, y1, x2, y2 = map(int, result.xyxy[0])

                # PINTAR DE COLOR VERDE SI EL CABALLO SE ENCUENTRA DENTRO DEL RECINTO Y DE ROJO SI ESTA FUERA
                center_x = (x1 + x2) // 2
                color = (0, 255, 0) if center_x < width // 2 else (0, 0, 255)
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        out.write(frame)

cap.release()
out.release()





0: 384x640 1 truck, 1 horse, 1 cow, 155.6ms
Speed: 3.7ms preprocess, 155.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)



KeyboardInterrupt: 

#EJERCICIO 2
VELOCIDAD CABALLO

In [14]:
import cv2
from ultralytics import YOLO
import numpy as np

# CARGAMOS YOLOV8
model = YOLO('yolov8n.pt')


# CARGAMOS EL VIDEO
cap = cv2.VideoCapture('caballo.mp4')


# CREAR VIDEO PROCESADO (OUTPUT)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = 'caballo_procesado_velocidad.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
#-------------------------------------------------------------

# PASAMOS DE PIXELES A METROS PARA LA CALCULACIÓN DE LA VELOCIDAD Y CALCULAMOS CUANTO DURA UN FRAME
pix_a_m = 0.05
frame_tiempo = 1 / fps


pos_anterior = {} #GUARDAMOS LAS POSICIONES ANTERIORES PARA LA CALCULACIÓN DE LAS VELOCIDADES

#FUNCIÓN PARA CALCULAR LA VELOCIDAD
#DEVUELVE 0 SI LA POSICIÓN ANTERIOR ES IGUAL A LA ACTUAL.
def calcular_velocidad(cx, cy, pos_anterior, obj_id, frame_rate, pix_a_m):
    if obj_id in pos_anterior:
        prev_cx, prev_cy = pos_anterior[obj_id]
        dx = cx - prev_cx
        dy = cy - prev_cy
        distancia_pixeles = np.sqrt(dx**2 + dy**2)
        distancia_metros = distancia_pixeles * pix_a_m
        velocidad = distancia_metros / frame_rate
        return velocidad
    return 0

# PROCESAMIENTO
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # TRACKING
    results = model.track(frame, persist=True)

    # DETECCIÓN SOLO DE CABALLOS
    for result in results[0].boxes:
        label = model.names[int(result.cls)]
        if label == 'horse':
            x1, y1, x2, y2 = map(int, result.xyxy[0])
            centro_x = (x1 + x2) // 2
            centro_y = (y1 + y2) // 2

            # OBTENEMOS EL ID DEL CABALLO PROPORCIONADO POR YOLO
            obj_id = int(result.id)

            # CALCULAMOS LA VELOCIDAD MEDIANTE LA FUNCIÓN CREADA ANTERIORMENTE
            velocidad = calcular_velocidad(centro_x, centro_y, pos_anterior, obj_id, frame_tiempo, pix_a_m)

            # MOSTRAR VELOCIDAD EN EL VIDEO PROCESADO
            cv2.putText(frame, f"Velocidad: {int(velocidad)} m/s", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # ACTULIZAMOS LA POSICIÓN ANTERIOR
            pos_anterior[obj_id] = (centro_x, centro_y)

            # CUADRO DE TRACKLING
            color = (0, 255, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    out.write(frame)

cap.release()
out.release()



0: 384x640 1 truck, 1 horse, 1 cow, 254.0ms
Speed: 4.4ms preprocess, 254.0ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 cow, 265.0ms
Speed: 4.0ms preprocess, 265.0ms inference, 4.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 257.5ms
Speed: 6.3ms preprocess, 257.5ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 241.7ms
Speed: 5.4ms preprocess, 241.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 228.2ms
Speed: 5.5ms preprocess, 228.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 186.8ms
Speed: 5.4ms preprocess, 186.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 1 horse, 1 umbrella, 158.8ms
Speed: 5.1ms preprocess, 158.8ms inference, 1.5ms postprocess per image 